In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
directory = "user_features"
data = pd.read_csv(os.path.join(directory,"data.csv"))
series_id_column = "user_id"
time_column = "time"
X= data[series_id_column].drop_duplicates()
def _id_selection(ids):
    keys= ids.values.tolist()
    return pd.concat([data.loc[(data[series_id_column] == key).values] for key in keys],
        ignore_index=True)
X_train, X_test= train_test_split(X, test_size=0.2, random_state=1)
train_dataset = _id_selection(X_train)  
train_dataset.sort_values(time_column, inplace=True)
test_dataset = _id_selection(X_test)
test_dataset.sort_values(time_column, inplace=True)
train_dataset.to_csv(os.path.join(directory,"train.csv"))
test_dataset.to_csv(os.path.join(directory,"test.csv"))

In [27]:
# import pandas as pd
# test_dataset = pd.read_csv('user_features/data.csv')
# series_id_column = 'user_id'
# test_dataset.sort_values('user_id', inplace=True)

In [29]:
state_columns = ["VariableName.sbp", "VariableName.dbp", "VariableName.salt_intake", "VariableName.step_count", "VariableName.sleep_duration", "category"]
keys = []
for column in state_columns:
    for i in range(6):
        keys.append(column + "_" + str(i))
ret_d = {key:[] for key in keys}
ret_d["reward"] = []
ret_d["nudge"] = []
for _, df in test_dataset.groupby(series_id_column):
    # 8:14
    # 22:28
    state_column_mapping = {}
    for idx, column in enumerate(list(df.columns)):
        state_column_mapping[column] = idx

    for column in state_columns:
        for time in range(6):
            ret_d[column+"_"+str(time)].append(df.iloc[time+8,state_column_mapping[column]])
    ret_d["reward"].append(df.iloc[18, state_column_mapping["VariableName.sbp"]] - df.iloc[13, state_column_mapping["VariableName.sbp"]])
    ret_d["nudge"].append(df.iloc[14, state_column_mapping["nudge"]]-1)
    
    for column in state_columns:
        for time in range(6):
            ret_d[column+"_"+str(time)].append(df.iloc[time+22, state_column_mapping[column]])
    ret_d["reward"].append(df.iloc[32, state_column_mapping["VariableName.sbp"]] - df.iloc[27, state_column_mapping["VariableName.sbp"]])
    ret_d["nudge"].append(df.iloc[28, state_column_mapping["nudge"]]-1)

newdf = pd.DataFrame(ret_d)
newdf["sampling_policy_0"] = 0.5
newdf["sampling_policy_1"] = 0.5

In [34]:
newdf.to_csv("user_features/test_reward_predictor.csv")